In [ ]:
import os

import numpy as np

# For DataFrame object
import pandas as pd

# Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

# Text Vectorizing
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

# Train-test-split
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize


In [ ]:
df = pd.read_csv('test.csv')
df.head()

,text,label
0,в Новый год ОРХИДЕЯ пойдёт с чистой ПОП* о...,0
1,Какой у вас денежный сценарий Если у бизнесмен...,0
2,Москвича избили и порезали при попытке купить ...,-1
3,Свинина тушеная Йошкар-Олинский мясокомбинат `...,-1
4,id138736268|Анна тогда сделайте подарок себе с...,1


In [ ]:
df.size

18772

In [ ]:
df = df.replace(0, 0.5)

In [ ]:
df = df.replace(-1, 0)

In [ ]:
print(df.label[1])

0.5


In [ ]:
df = df.dropna()

In [ ]:
target = np.array(df['label'].astype('uint8'))
target[:5]

array([0, 0, 0, 0, 1], dtype=uint8)

In [ ]:
tokenizer = Tokenizer(num_words=30000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
                      lower=True, 
                      split=' ', 
                      char_level=False)

In [ ]:
tokenizer.fit_on_texts(df['text'])
matrix = tokenizer.texts_to_matrix(df['text'], mode='count')
matrix.shape

(9386, 30000)

In [ ]:
def get_model():
    
    model = Sequential()
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', 
                  loss='mse',
                  metrics=['accuracy'])
    
    return model

In [ ]:
X = normalize(matrix)
y = target

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.2)

X_train.shape, y_train.shape

((7508, 30000), (7508,))

In [ ]:
model = get_model()

history = model.fit(X_train, 
                    y_train, 
                    epochs=150, 
                    batch_size=500,
                    validation_data=(X_test, y_test))

history

Train on 7508 samples, validate on 1878 samples
Epoch 1/150
7508/7508 [==============================] - 3s 372us/step - loss: 0.2420 - accuracy: 0.6747 - val_loss: 0.2310 - val_accuracy: 0.6672
Epoch 2/150
7508/7508 [==============================] - 3s 335us/step - loss: 0.2215 - accuracy: 0.6770 - val_loss: 0.2149 - val_accuracy: 0.6672
Epoch 3/150
7508/7508 [==============================] - 3s 334us/step - loss: 0.2067 - accuracy: 0.6770 - val_loss: 0.2048 - val_accuracy: 0.6672
Epoch 4/150
7508/7508 [==============================] - 3s 336us/step - loss: 0.1909 - accuracy: 0.6770 - val_loss: 0.1916 - val_accuracy: 0.6672
Epoch 5/150
7508/7508 [==============================] - 3s 335us/step - loss: 0.1739 - accuracy: 0.6770 - val_loss: 0.1807 - val_accuracy: 0.6672
Epoch 6/150
7508/7508 [==============================] - 3s 338us/step - loss: 0.1546 - accuracy: 0.6770 - val_loss: 0.1710 - val_accuracy: 0.6672
Epoch 7/150
7508/7508 [==============================] - 3s 333us/step

In [ ]:
model.save("model.pt")